In [ ]:
import json
import pandas as pd

with open('./train.json', 'r', encoding='UTF8') as f:
    data = json.load(f)
df = pd.json_normalize(data)

In [ ]:
df.describe()

In [ ]:
# Sort By like_cnt and update_date
# give weight to 'like_cnt'
df.sort_values(by=['like_cnt', 'updt_date'], ascending=False, inplace=True)

In [ ]:
new_df = df.iloc[:1000]
for idx, row in new_df.iterrows():
    print(row['tags'], row['like_cnt'])